In [1]:
from datetime import date, time
#import xlsxwriter
import pandas as pd
import numpy as np
import json
import re

import os 

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)


In [2]:
in_dir = "/Users/dipanjanpaul/Winpra/app/mkt_data/"
fl = os.listdir(in_dir)

In [6]:
infile = in_dir + fl[0]
worksheet = 0

In [7]:
wb_sheets = pd.ExcelFile(infile).sheet_names
wb_sheets

['T12-2017']

In [10]:
df = pd.read_excel(infile, sheet_name=wb_sheets[0], header=0)
df.head()

,Unnamed: 0,Jan 2017,Feb 2017,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
0,INCOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RENTAL INCOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gross Potential Rent,273574.0,273574.0,273574.00,307974.00,307974.00,307974.00,307974.00,307974.00,312056.44750,316138.8950,320221.34250,324303.790
4,Gain/Loss to Lease,27130.0,28626.0,29958.67,602.68,4382.15,8016.07,8709.37,11373.29,8853.20125,6333.1125,3813.02375,1292.935


In [20]:
ix = df.apply(lambda x: sum(x.isnull()) == 13, axis=1)
ix = np.where(ix != True)[0]
df.iloc[ix, :].head()

,Unnamed: 0,Jan 2017,Feb 2017,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
0,INCOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RENTAL INCOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gross Potential Rent,273574.0,273574.0,273574.00,307974.00,307974.00,307974.00,307974.00,307974.00,312056.44750,316138.8950,320221.34250,324303.790
4,Gain/Loss to Lease,27130.0,28626.0,29958.67,602.68,4382.15,8016.07,8709.37,11373.29,8853.20125,6333.1125,3813.02375,1292.935
5,Rent,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00000,0.0000,0.00000,0.000


In [109]:
df = pd.read_excel(infile, sheet_name=worksheet, header=0)

ix = df.apply(lambda x: sum(x.isnull()) == 13, axis=1)
ix = np.where(ix != True)[0]
df = df.iloc[ix, :]

x = ["Catg_SubCatg"]
x.extend(df.columns[1:])
x

df.columns = x

df.Catg_SubCatg = [x.strip() for x in df.Catg_SubCatg]

df = df[-df.Catg_SubCatg.str.startswith(tuple(["TOTAL", "NET", "INCOME", "EXPENSE"]))]

df.reset_index(drop=True, inplace=True)
df.head(2)

,Catg_SubCatg,Jan 2017,Feb 2017,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
0,RENTAL INCOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gross Potential Rent,273574.0,273574.0,273574.0,307974.0,307974.0,307974.0,307974.0,307974.0,312056.4475,316138.895,320221.3425,324303.79


In [110]:
df.shape

(218, 13)

In [111]:
df.tail(2)

,Catg_SubCatg,Jan 2017,Feb 2017,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
216,SMP - Electric Upgrades,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217,SMP - Roof Construction,0.0,0.0,0.0,0.0,0.0,0.0,1250.0,0.0,0.0,0.0,0.0,0.0


In [112]:
ix = np.where(df.iloc[:, 1:].apply(lambda x: sum(x.isnull())==12, axis=1))[0]

catg_map = pd.DataFrame(df.iloc[ix,0])
catg_map['i_start'] = catg_map.index

i_end = list(catg_map.index[1:].values)
i_end.append(df.shape[0])
catg_map['i_end'] = i_end

catg_map['repeat_cnt'] = catg_map.i_end - catg_map.i_start
catg_map.reset_index(drop=True, inplace=True)

#catg_map

In [113]:
x = catg_map.apply(lambda x: list(np.repeat(x[0].strip(), x[3])), axis=1)
catg = [itm for sublist in x for itm in sublist]
len(catg)

218

In [114]:
df['Category'] = catg
df.drop(index=ix, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(2)

,Catg_SubCatg,Jan 2017,Feb 2017,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017,Category
0,Gross Potential Rent,273574.0,273574.0,273574.00,307974.00,307974.00,307974.00,307974.00,307974.00,312056.44750,316138.8950,320221.34250,324303.790,RENTAL INCOME
1,Gain/Loss to Lease,27130.0,28626.0,29958.67,602.68,4382.15,8016.07,8709.37,11373.29,8853.20125,6333.1125,3813.02375,1292.935,RENTAL INCOME


In [116]:
df.Category.value_counts()

REPAIRS & MAINTENANCE EXPENSE     38
GENERAL & ADMINISTRATIVE          33
ANCILLARY INCOME                  30
CAPITAL EXPENDITURES              29
RENTAL INCOME                     20
ADVERTISING & MARKETING           13
PAYROLL COSTS                     12
DEBT INTEREST                     10
UTILITIES                          6
SECURITY SERVICE                   3
TAXES                              3
PROFESSIONAL & ACCOUNTING FEES     3
MANAGEMENT FEES                    2
INSURANCE                          2
Name: Category, dtype: int64

In [118]:
df.columns = ['SubCategory', '01_JAN', '02_FEB', '03_MAR', '04_APR', '05_MAY', '06_JUN', '07_JUL',
              '08_AUG', '09_SEP', '10_OCT', '11_NOV', '12_DEC', 'Category']

df = df[['Category', 'SubCategory', '01_JAN', '02_FEB', '03_MAR', '04_APR', '05_MAY', '06_JUN', '07_JUL',
              '08_AUG', '09_SEP', '10_OCT', '11_NOV', '12_DEC']]

df.head()

,Category,SubCategory,01_JAN,02_FEB,03_MAR,04_APR,05_MAY,06_JUN,07_JUL,08_AUG,09_SEP,10_OCT,11_NOV,12_DEC
0,RENTAL INCOME,Gross Potential Rent,273574.00,273574.00,273574.00,307974.00,307974.00,307974.00,307974.00,307974.00,312056.44750,316138.8950,320221.34250,324303.790
1,RENTAL INCOME,Gain/Loss to Lease,27130.00,28626.00,29958.67,602.68,4382.15,8016.07,8709.37,11373.29,8853.20125,6333.1125,3813.02375,1292.935
2,RENTAL INCOME,Rent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00000,0.0000,0.00000,0.000
3,RENTAL INCOME,Change in Delinquency,-6128.79,-6986.13,-4738.74,-9137.83,-7975.68,-5259.50,-3046.74,-2167.94,-1896.94750,-1625.9550,-1354.96250,-1083.970
4,RENTAL INCOME,Change in Prepaid Rent,-623.90,879.49,-698.59,3101.64,9489.56,-12913.76,435.00,-687.00,-601.12500,-515.2500,-429.37500,-343.500


In [119]:
final_df = df.melt(id_vars=['Category', 'SubCategory'],
                  value_vars=['01_JAN', '02_FEB', '03_MAR', '04_APR', '05_MAY', '06_JUN', 
                              '07_JUL', '08_AUG', '09_SEP', '10_OCT', '11_NOV'])

final_df.SubCategory = [x.strip() for x in final_df.SubCategory]

final_df.head()

,Category,SubCategory,variable,value
0,RENTAL INCOME,Gross Potential Rent,01_JAN,273574.00
1,RENTAL INCOME,Gain/Loss to Lease,01_JAN,27130.00
2,RENTAL INCOME,Rent,01_JAN,0.00
3,RENTAL INCOME,Change in Delinquency,01_JAN,-6128.79
4,RENTAL INCOME,Change in Prepaid Rent,01_JAN,-623.90


In [121]:
#final_df[final_df.Category == 'UTILITIES']

In [122]:
#final_df.variable.value_counts()

In [11]:
def process_yearly_file(infile, worksheet):
    #Part 1
    df = pd.read_excel(infile, sheet_name=worksheet, header=0)

    ix = df.apply(lambda x: sum(x.isnull()) == 13, axis=1)
    ix = np.where(ix != True)[0]
    df = df.iloc[ix, :]

    x = ["Catg_SubCatg"]
    x.extend(df.columns[1:])
    x

    df.columns = x

    df.Catg_SubCatg = [x.strip() for x in df.Catg_SubCatg]

    df = df[-df.Catg_SubCatg.str.startswith(tuple(["TOTAL", "NET", "INCOME", "EXPENSE"]))]

    df.reset_index(drop=True, inplace=True)  
    
    #Part 2
    #print(df.head())
    ix = np.where(df.iloc[:, 1:].apply(lambda x: sum(x.isnull())==12, axis=1))[0]

    catg_map = pd.DataFrame(df.iloc[ix,0])
    catg_map['i_start'] = catg_map.index

    i_end = list(catg_map.index[1:].values)
    i_end.append(df.shape[0])
    catg_map['i_end'] = i_end

    catg_map['repeat_cnt'] = catg_map.i_end - catg_map.i_start
    catg_map.reset_index(drop=True, inplace=True)
    
    x = catg_map.apply(lambda x: list(np.repeat(x[0].strip(), x[3])), axis=1)
    catg = [itm for sublist in x for itm in sublist]

    df['Category'] = catg
    df.drop(index=ix, inplace=True)
    df.reset_index(drop=True, inplace=True)

    #Part 3
    df.columns = ['SubCategory', '01_JAN', '02_FEB', '03_MAR', '04_APR', '05_MAY', '06_JUN', '07_JUL',
                  '08_AUG', '09_SEP', '10_OCT', '11_NOV', '12_DEC', 'Category']

    df = df[['Category', 'SubCategory', '01_JAN', '02_FEB', '03_MAR', '04_APR', '05_MAY', '06_JUN', '07_JUL',
                  '08_AUG', '09_SEP', '10_OCT', '11_NOV', '12_DEC']]

    
    #Part 4
    df = df.melt(id_vars=['Category', 'SubCategory'],
                  value_vars=['01_JAN', '02_FEB', '03_MAR', '04_APR', '05_MAY', '06_JUN', 
                              '07_JUL', '08_AUG', '09_SEP', '10_OCT', '11_NOV', '12_DEC'])
    
    
    df.columns = ['Category', 'SubCategory', 'MM', 'Value']
    df.SubCategory = [x.strip() for x in df.SubCategory]

    df['P_Year'] = worksheet
    return(df)

In [12]:
in_dir = "/Users/dipanjanpaul/Winpra/app/mkt_data/"
fl = os.listdir(in_dir)
fl

['T12-2017.xlsx',
 'T12-2016.xlsx',
 '~$test_t12_data.xlsx',
 'mkt_analysis.json',
 'test_t12_data.xlsx',
 'proc_t12_data.csv',
 'T12-2015.xlsx',
 'mkt_analysis.csv',
 'T12-2018.xlsx']

In [187]:
final = []
for itm in fl:
    if itm.split('.')[1] == 'xlsx' and itm.split('.')[0].startswith('T12'):
        infile = in_dir + itm
        worksheet = itm.split('.')[0]
        print(worksheet)
        final.append(process_yearly_file(infile, worksheet))
   

T12-2017
T12-2016
T12-2015
T12-2018


In [188]:
final_df = pd.concat(final, axis=0)

In [192]:
final_df.to_csv(in_dir + "mkt_analysis.csv", header=True, index=False)

In [13]:
inp = "/Users/dipanjanpaul/Winpra/app/mkt_data/test_t12_data.xlsx"

wb_sheets = pd.ExcelFile(inp).sheet_names

    #print(wb_sheets)

final = []
for itm in wb_sheets:
    print(itm)
    final.append(process_yearly_file(inp, itm))

final_df = pd.concat(final, axis=0)


T12-2015
T12-2016
T12-2017
T12-2018


In [15]:
final_df[final_df.SubCategory == "Water & Sewer Income"]

,Category,SubCategory,MM,Value,P_Year
22,ANCILLARY INCOME,Water & Sewer Income,01_JAN,0.0000,T12-2015
224,ANCILLARY INCOME,Water & Sewer Income,02_FEB,16944.1900,T12-2015
426,ANCILLARY INCOME,Water & Sewer Income,03_MAR,19971.1900,T12-2015
628,ANCILLARY INCOME,Water & Sewer Income,04_APR,20498.2100,T12-2015
830,ANCILLARY INCOME,Water & Sewer Income,05_MAY,20330.4800,T12-2015
1032,ANCILLARY INCOME,Water & Sewer Income,06_JUN,22594.5700,T12-2015
1234,ANCILLARY INCOME,Water & Sewer Income,07_JUL,21381.4800,T12-2015
1436,ANCILLARY INCOME,Water & Sewer Income,08_AUG,24329.8300,T12-2015
1638,ANCILLARY INCOME,Water & Sewer Income,09_SEP,27798.2700,T12-2015
1840,ANCILLARY INCOME,Water & Sewer Income,10_OCT,21778.3200,T12-2015


In [9]:
final_df.head()

,Category,SubCategory,MM,Value,P_Year
0,RENTAL INCOME,Gross Potential Rent,01_JAN,0.00,T12-2015
1,RENTAL INCOME,Gain/Loss to Lease,01_JAN,0.00,T12-2015
2,RENTAL INCOME,Rent,01_JAN,172777.68,T12-2015
3,RENTAL INCOME,Change in Delinquency,01_JAN,0.00,T12-2015
4,RENTAL INCOME,Change in Prepaid Rent,01_JAN,246.89,T12-2015


In [194]:
pd.crosstab(final_df.P_Year, final_df.Category)

Category,ADVERTISING & MARKETING,ANCILLARY INCOME,CAPITAL EXPENDITURES,DEBT INTEREST,GENERAL & ADMINISTRATIVE,INSURANCE,MANAGEMENT FEES,NON OPERATING/NON RECURRING EXPENSES,PAYROLL COSTS,PROFESSIONAL & ACCOUNTING FEES,PY EXPENSE ADJUSTMENTS,RENTAL INCOME,REPAIRS & MAINTENANCE EXPENSE,SECURITY SERVICE,TAXES,UTILITIES
P_Year,,,,,,,,,,,,,,,,
T12-2015,156,360,348,60,396,24,24,24,144,36,12,240,456,36,36,72
T12-2016,156,360,348,60,396,24,24,24,144,36,12,240,456,36,36,72
T12-2017,156,360,348,60,396,24,24,24,144,36,12,240,456,36,36,72
T12-2018,156,360,348,60,396,24,24,24,144,36,12,240,456,36,36,72


In [191]:
final_df[final_df.SubCategory == "CAPITAL EXPENDITURES"]

,Category,SubCategory,MM,Value,P_Year


In [162]:
x = final_df.groupby(['Category', 'SubCategory', 'P_Year']) \
            .apply(lambda x: x[['MM', 'Value']].to_dict('records')) \
            .reset_index() \
            .rename(columns={0:'Monthly_Numbers'}) \
            .to_json(orient='records')

with open(in_dir + "mkt_analysis.json", 'w') as f:
    json.dump(x, f)

In [408]:
type(x)

str

In [378]:
y = json.loads(x)
y[0]

{'Category': 'ADVERTISING & MARKETING',
 'SubCategory': 'Adv & Mkting - 3rd Party Advertising',
 'P_Year': 'T12-2015',
 'Monthly_Numbers': [{'MM': '01_JAN', 'Value': 0.0},
  {'MM': '02_FEB', 'Value': 50.0},
  {'MM': '03_MAR', 'Value': 1275.8},
  {'MM': '04_APR', 'Value': 1858.57},
  {'MM': '05_MAY', 'Value': 2829.6},
  {'MM': '06_JUN', 'Value': 2552.8},
  {'MM': '07_JUL', 'Value': 2702.65},
  {'MM': '08_AUG', 'Value': 2516.44},
  {'MM': '09_SEP', 'Value': 825.65},
  {'MM': '10_OCT', 'Value': 3781.65},
  {'MM': '11_NOV', 'Value': 1627.85}]}

In [382]:
y[2].get('Category')

'ADVERTISING & MARKETING'

In [383]:
len(y)

286

In [402]:
d = {}
for key, value in final_df[['Category', 'SubCategory', 'MM', 'Value', 'P_Year']].groupby(['Category']):
    
    i_dict = value.groupby(['SubCategory', 'P_Year']) \
            .apply(lambda x: x[['MM', 'Value']].to_dict('records')) \
            .reset_index() \
            .rename(columns={0:'Monthly_Numbers'}) \
            .to_json(orient='records')
    
    d[key] = i_dict
    
with open('/Users/dipanjanpaul/Winpra/dash/data/mkt_analysis_1.json', 'w') as f:
    json.dump(d, f)

In [407]:
type(d)

dict

In [409]:
final_df.Category.value_counts()

REPAIRS & MAINTENANCE EXPENSE           693
CAPITAL EXPENDITURES                    572
GENERAL & ADMINISTRATIVE                484
ANCILLARY INCOME                        374
RENTAL INCOME                           264
PAYROLL COSTS                           198
UTILITIES                               121
ADVERTISING & MARKETING                 110
DEBT INTEREST                            77
SECURITY SERVICE                         55
TAXES                                    44
PROFESSIONAL & ACCOUNTING FEES           44
MANAGEMENT FEES                          33
INSURANCE                                33
NON OPERATING/NON RECURRING EXPENSES     22
PY EXPENSE ADJUSTMENTS                   22
Name: Category, dtype: int64

In [411]:
final_df[final_df.Category == "RENTAL INCOME"].SubCategory.value_counts()

Gross Potential Rent                      22
Gain/Loss to Lease                        22
Change in Delinquency                     22
Change in Prepaid Rent                    22
Vacancy                                   22
Less: Recurring Concessions               22
Less: Move-in Specials                    22
Less: Referral Concessions                22
Less: Past Tenant Collections             22
Concession Chargebacks                    22
Rent                                      11
Other Discount Units                      11
Less: Renewal Concessions                 11
Less: Write-off Concession Charge back    11
Name: SubCategory, dtype: int64